In [ ]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [ ]:
#export
from exp.nb_11 import *

## Serializing the model

In [ ]:
path = datasets.untar_data(datasets.URLs.IMAGEWOOF_160)

In [ ]:
size = 128
bs = 64

tfms = [make_rgb, RandomResizedCrop(128,scale=(0.35,1)), np_to_float, PilRandomFlip()]
val_tfms = [make_rgb, CenterCrop(size), np_to_float]
il = ImageList.from_files(path, tfms=tfms)
sd = SplitData.split_by_func(il, partial(grandparent_splitter, valid_name='val'))
ll = label_by_func(sd, parent_labeler, proc_y=CategoryProcessor())
ll.valid.x.tfms = val_tfms
data = ll.to_databunch(bs, c_in=3, c_out=10, num_workers=8)

In [ ]:
loss_func = LabelSmoothingCrossEntropy()
opt_func = adam_opt(mom=0.9, mom_sqr=0.99, eps=1e-6, wd=1e-2)

In [ ]:
learn = cnn_learner(xresnet18, data, loss_func, opt_func, norm=norm_imagenette)

In [ ]:
def sched_1cycle(lr, pct_start=0.3, mom_start=0.95, mom_mid=0.85, mom_end=0.95):
    phases = create_phases(pct_start)
    sched_lr  = combine_scheds(phases, cos_1cycle_anneal(lr/10., lr, lr/1e5))
    sched_mom = combine_scheds(phases, cos_1cycle_anneal(mom_start, mom_mid, mom_end))
    return [ParamScheduler('lr', sched_lr),
            ParamScheduler('mom', sched_mom)]

In [ ]:
lr = 3e-3
pct_start = 0.5
cbsched = sched_1cycle(lr, pct_start)

In [ ]:
learn.fit(5, cbsched)

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time
0,2.126259,0.251405,2.144876,0.260000,00:11
1,1.988209,0.320700,2.759073,0.232000,00:11
2,1.821927,0.402280,1.998702,0.344000,00:11
3,1.644655,0.493014,1.716803,0.422000,00:11
4,1.472529,0.574675,1.475785,0.564000,00:11


In [ ]:
st = learn.model.state_dict()

In [ ]:
type(st)

collections.OrderedDict

In [ ]:
', '.join(st.keys())

'0.0.weight, 0.1.weight, 0.1.bias, 0.1.running_mean, 0.1.running_var, 0.1.num_batches_tracked, 1.0.weight, 1.1.weight, 1.1.bias, 1.1.running_mean, 1.1.running_var, 1.1.num_batches_tracked, 2.0.weight, 2.1.weight, 2.1.bias, 2.1.running_mean, 2.1.running_var, 2.1.num_batches_tracked, 4.0.convs.0.0.weight, 4.0.convs.0.1.weight, 4.0.convs.0.1.bias, 4.0.convs.0.1.running_mean, 4.0.convs.0.1.running_var, 4.0.convs.0.1.num_batches_tracked, 4.0.convs.1.0.weight, 4.0.convs.1.1.weight, 4.0.convs.1.1.bias, 4.0.convs.1.1.running_mean, 4.0.convs.1.1.running_var, 4.0.convs.1.1.num_batches_tracked, 4.1.convs.0.0.weight, 4.1.convs.0.1.weight, 4.1.convs.0.1.bias, 4.1.convs.0.1.running_mean, 4.1.convs.0.1.running_var, 4.1.convs.0.1.num_batches_tracked, 4.1.convs.1.0.weight, 4.1.convs.1.1.weight, 4.1.convs.1.1.bias, 4.1.convs.1.1.running_mean, 4.1.convs.1.1.running_var, 4.1.convs.1.1.num_batches_tracked, 5.0.convs.0.0.weight, 5.0.convs.0.1.weight, 5.0.convs.0.1.bias, 5.0.convs.0.1.running_mean, 5.0.convs

In [ ]:
st['10.bias']

tensor([ 0.0041, -0.0085, -0.0073,  0.0080, -0.0067, -0.0015, -0.0031,  0.0071,
         0.0025,  0.0081], device='cuda:0')

In [ ]:
mdl_path = path/'models'
mdl_path.mkdir(exist_ok=True)

It's also possible to save the whole model, including the architecture, but it gets quite fiddly and we don't recommend it. Instead, just save the parameters, and recreate the model directly.

In [ ]:
torch.save(st, mdl_path/'iw5')

## Custom head

In [ ]:
pets = datasets.untar_data(datasets.URLs.PETS)

In [ ]:
pets.ls()

[PosixPath('/home/jhoward/.fastai/data/oxford-iiit-pet/models'),
 PosixPath('/home/jhoward/.fastai/data/oxford-iiit-pet/annotations'),
 PosixPath('/home/jhoward/.fastai/data/oxford-iiit-pet/images')]

In [ ]:
pets_path = pets/'images'

In [ ]:
tfms = [make_rgb, RandomResizedCrop(128,scale=(0.35,1)), np_to_float, PilRandomFlip()]
il = ImageList.from_files(pets_path, tfms=tfms)

In [ ]:
il

ImageList (7390 items)
[PosixPath('/home/jhoward/.fastai/data/oxford-iiit-pet/images/yorkshire_terrier_187.jpg'), PosixPath('/home/jhoward/.fastai/data/oxford-iiit-pet/images/german_shorthaired_112.jpg'), PosixPath('/home/jhoward/.fastai/data/oxford-iiit-pet/images/american_bulldog_216.jpg'), PosixPath('/home/jhoward/.fastai/data/oxford-iiit-pet/images/basset_hound_124.jpg'), PosixPath('/home/jhoward/.fastai/data/oxford-iiit-pet/images/basset_hound_19.jpg'), PosixPath('/home/jhoward/.fastai/data/oxford-iiit-pet/images/pug_156.jpg'), PosixPath('/home/jhoward/.fastai/data/oxford-iiit-pet/images/Ragdoll_138.jpg'), PosixPath('/home/jhoward/.fastai/data/oxford-iiit-pet/images/Sphynx_128.jpg'), PosixPath('/home/jhoward/.fastai/data/oxford-iiit-pet/images/samoyed_175.jpg'), PosixPath('/home/jhoward/.fastai/data/oxford-iiit-pet/images/great_pyrenees_128.jpg')...]
Path: /home/jhoward/.fastai/data/oxford-iiit-pet/images

In [ ]:
#export
def random_splitter(fn, p_valid): return random.random() < p_valid

In [ ]:
random.seed(42)

In [ ]:
sd = SplitData.split_by_func(il, partial(random_splitter, p_valid=0.1))

In [ ]:
sd

SplitData
Train: ImageList (6667 items)
[PosixPath('/home/jhoward/.fastai/data/oxford-iiit-pet/images/yorkshire_terrier_187.jpg'), PosixPath('/home/jhoward/.fastai/data/oxford-iiit-pet/images/american_bulldog_216.jpg'), PosixPath('/home/jhoward/.fastai/data/oxford-iiit-pet/images/basset_hound_124.jpg'), PosixPath('/home/jhoward/.fastai/data/oxford-iiit-pet/images/basset_hound_19.jpg'), PosixPath('/home/jhoward/.fastai/data/oxford-iiit-pet/images/pug_156.jpg'), PosixPath('/home/jhoward/.fastai/data/oxford-iiit-pet/images/Ragdoll_138.jpg'), PosixPath('/home/jhoward/.fastai/data/oxford-iiit-pet/images/samoyed_175.jpg'), PosixPath('/home/jhoward/.fastai/data/oxford-iiit-pet/images/boxer_32.jpg'), PosixPath('/home/jhoward/.fastai/data/oxford-iiit-pet/images/Bombay_22.jpg'), PosixPath('/home/jhoward/.fastai/data/oxford-iiit-pet/images/pug_57.jpg')...]
Path: /home/jhoward/.fastai/data/oxford-iiit-pet/images
Valid: ImageList (723 items)
[PosixPath('/home/jhoward/.fastai/data/oxford-iiit-pet/im

In [ ]:
n = il.items[0].name

In [ ]:
re.findall(r'^(.*)_\d+.jpg$', n)[0]

'yorkshire_terrier'

In [ ]:
def pet_labeler(fn): return re.findall(r'^(.*)_\d+.jpg$', fn.name)[0]

In [ ]:
proc = CategoryProcessor()

In [ ]:
ll = label_by_func(sd, pet_labeler, proc_y=proc)

In [ ]:
', '.join(proc.vocab)

'yorkshire_terrier, american_bulldog, basset_hound, pug, Ragdoll, samoyed, boxer, Bombay, wheaten_terrier, english_setter, Siamese, British_Shorthair, Birman, american_pit_bull_terrier, great_pyrenees, pomeranian, saint_bernard, scottish_terrier, Abyssinian, Maine_Coon, havanese, chihuahua, german_shorthaired, Sphynx, beagle, Russian_Blue, keeshond, Bengal, japanese_chin, shiba_inu, newfoundland, staffordshire_bull_terrier, leonberger, Egyptian_Mau, Persian, english_cocker_spaniel, miniature_pinscher'

In [ ]:
ll.valid.x.tfms = val_tfms

In [ ]:
c_out = len(proc.vocab)

In [ ]:
data = ll.to_databunch(bs, c_in=3, c_out=c_out, num_workers=8)

In [ ]:
learn = cnn_learner(xresnet18, data, loss_func, opt_func, norm=norm_imagenette)

In [ ]:
learn.fit(5, cbsched)

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time
0,3.452545,0.092245,3.408346,0.088520,00:07
1,3.258923,0.138443,3.972067,0.098202,00:07
2,3.051906,0.195440,3.658121,0.116183,00:07
3,2.757764,0.283186,2.954207,0.248963,00:07
4,2.488618,0.378281,2.573904,0.347165,00:07


In [ ]:
learn = cnn_learner(xresnet18, data, loss_func, opt_func, c_out=10, norm=norm_imagenette)

In [ ]:
st = torch.load(mdl_path/'iw5')

In [ ]:
m = learn.model

In [ ]:
m.load_state_dict(st)

In [ ]:
cut = next(i for i,o in enumerate(m.children()) if isinstance(o,nn.AdaptiveAvgPool2d))
m_cut = m[:cut]

In [ ]:
xb,yb = get_batch(data.valid_dl, learn)

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time


In [ ]:
pred = m_cut(xb)

In [ ]:
pred.shape

torch.Size([128, 512, 4, 4])

In [ ]:
ni = pred.shape[1]

In [ ]:
#export
class AdaptiveConcatPool2d(nn.Module):
    def __init__(self, sz=1):
        super().__init__()
        self.output_size = sz
        self.ap = nn.AdaptiveAvgPool2d(sz)
        self.mp = nn.AdaptiveMaxPool2d(sz)
    def forward(self, x): return torch.cat([self.mp(x), self.ap(x)], 1)

In [ ]:
nh = 40

m_new = nn.Sequential(
    m_cut, AdaptiveConcatPool2d(), Flatten(),
    nn.Linear(ni*2, data.c_out))

In [ ]:
learn.model = m_new

In [ ]:
learn.fit(5, cbsched)

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time
0,3.113310,0.175041,3.018431,0.215768,00:07
1,2.733775,0.284236,3.053549,0.210235,00:07
2,2.567899,0.343783,3.010797,0.218534,00:07
3,2.235002,0.447728,2.384175,0.398340,00:07
4,1.947652,0.559772,2.067362,0.497925,00:07


## adapt_model

In [ ]:
def adapt_model(learn, data):
    cut = next(i for i,o in enumerate(learn.model.children())
               if isinstance(o,nn.AdaptiveAvgPool2d))
    m_cut = learn.model[:cut]
    xb,yb = get_batch(data.valid_dl, learn)
    pred = m_cut(xb)
    ni = pred.shape[1]
    m_new = nn.Sequential(
        m_cut, AdaptiveConcatPool2d(), Flatten(),
        nn.Linear(ni*2, data.c_out))
    learn.model = m_new

In [ ]:
learn = cnn_learner(xresnet18, data, loss_func, opt_func, c_out=10, norm=norm_imagenette)
learn.model.load_state_dict(torch.load(mdl_path/'iw5'))

In [ ]:
adapt_model(learn, data)

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time


In [ ]:
for p in learn.model[0].parameters(): p.requires_grad_(False)

In [ ]:
learn.fit(2, sched_1cycle(1e-2, 0.5))

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time
0,3.657594,0.159442,5.360384,0.084371,00:05
1,3.368228,0.258737,2.679673,0.316736,00:05


In [ ]:
for p in learn.model[0].parameters(): p.requires_grad_(False)

In [ ]:
learn.fit(5, cbsched, reset_opt=True)

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time
0,2.557824,0.355032,2.642628,0.315353,00:05
1,2.678106,0.319334,2.884204,0.254495,00:05
2,2.730905,0.321734,2.737496,0.304288,00:05
3,2.567242,0.354582,2.594845,0.337483,00:05
4,2.427068,0.402430,2.480342,0.380360,00:05


## Batch norm transfer

In [ ]:
learn = cnn_learner(xresnet18, data, loss_func, opt_func, c_out=10, norm=norm_imagenette)
learn.model.load_state_dict(torch.load(mdl_path/'iw5'))
adapt_model(learn, data)

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time


In [ ]:
def apply_mod(m, f):
    f(m)
    for l in m.children(): apply_mod(l, f)

def set_grad(m, b):
    if isinstance(m, (nn.Linear,nn.BatchNorm2d)): return
    if hasattr(m, 'weight'):
        for p in m.parameters(): p.requires_grad_(b)

In [ ]:
apply_mod(learn.model, partial(set_grad, b=False))

In [ ]:
learn.fit(2, sched_1cycle(1e-2, 0.5))

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time
0,3.218999,0.181491,2.892865,0.230982,00:05
1,2.638681,0.315734,2.536313,0.351314,00:05


In [ ]:
apply_mod(learn.model, partial(set_grad, b=True))

In [ ]:
learn.fit(5, cbsched, reset_opt=True)

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time
0,2.425208,0.397030,2.633618,0.320885,00:07
1,2.393387,0.402880,2.971740,0.261411,00:07
2,2.314925,0.427929,2.767396,0.294606,00:07
3,2.072245,0.515974,2.166988,0.474412,00:07
4,1.837533,0.608070,1.988008,0.547718,00:07


Pytorch already has an `apply` method we can use:

In [ ]:
learn.model.apply(partial(set_grad, b=False));

## Discriminative LR and param groups

In [ ]:
learn = cnn_learner(xresnet18, data, loss_func, opt_func, c_out=10, norm=norm_imagenette)

In [ ]:
learn.model.load_state_dict(torch.load(mdl_path/'iw5'))
adapt_model(learn, data)

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time


In [ ]:
def bn_splitter(m):
    def _bn_splitter(l, g1, g2):
        if isinstance(l, (nn.Linear,nn.BatchNorm2d)): g2 += l.parameters()
        elif hasattr(l, 'weight'): g1 += l.parameters()
        for ll in l.children(): _bn_splitter(ll, g1, g2)
        
    g1,g2 = [],[]
    _bn_splitter(m[0], g1, g2)
    
    g2 += m[1:].parameters()
    return g1,g2

In [ ]:
a,b = bn_splitter(learn.model)

In [ ]:
test_eq(len(a)+len(b), len(list(m.parameters())))

In [ ]:
Learner.ALL_CBS

{'after_backward',
 'after_batch',
 'after_cancel_batch',
 'after_cancel_epoch',
 'after_cancel_train',
 'after_epoch',
 'after_fit',
 'after_loss',
 'after_pred',
 'after_step',
 'begin_batch',
 'begin_epoch',
 'begin_fit',
 'begin_validate'}

In [ ]:
#export
from types import SimpleNamespace
cb_types = SimpleNamespace(**{o:o for o in Learner.ALL_CBS})

In [ ]:
cb_types.after_backward

'after_backward'

In [ ]:
#export
class DebugCallback(Callback):
    _order = 999
    def __init__(self, cb_name, f=None): self.cb_name,self.f = cb_name,f
    def __call__(self, cb_name):
        if cb_name==self.cb_name:
            if self.f: self.f(self.run)
            else:      set_trace()

In [ ]:
def scheds_1cycle(lrs, pct_start=0.3, mom_start=0.95, mom_mid=0.85, mom_end=0.95):
    phases = create_phases(pct_start)
    sched_lr  = [combine_scheds(phases, cos_1cycle_anneal(lr/10., lr, lr/1e5))
                 for lr in lrs]
    sched_mom = combine_scheds(phases, cos_1cycle_anneal(mom_start, mom_mid, mom_end))
    return [ParamScheduler('lr', sched_lr),
            ParamScheduler('mom', sched_mom)]

In [ ]:
disc_lr_sched = scheds_1cycle([1e-3,1e-2], 0.5)

In [ ]:
learn = cnn_learner(xresnet18, data, loss_func, opt_func,
                    c_out=10, norm=norm_imagenette, splitter=bn_splitter)

In [ ]:
learn.model.load_state_dict(torch.load(mdl_path/'iw5'))
adapt_model(learn, data)

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time


In [ ]:
def _print_det(o): 
    print (len(o.opt.param_groups), o.opt.hypers)
    raise CancelTrainException()

learn.fit(1, disc_lr_sched + [DebugCallback(cb_types.after_batch, _print_det)])

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time


2 [{'mom': 0.9, 'mom_sqr': 0.99, 'eps': 1e-06, 'wd': 0.01, 'lr': 0.0001000000000000017, 'sqr_mom': 0.99}, {'mom': 0.9, 'mom_sqr': 0.99, 'eps': 1e-06, 'wd': 0.01, 'lr': 0.001000000000000017, 'sqr_mom': 0.99}]


In [ ]:
learn.fit(5, disc_lr_sched)

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time


TypeError: zip argument #1 must support iteration

In [ ]:
%debug

> /data1/jhoward/git/fastai_docs/dev_course/dl2/exp/nb_09.py(40)set_param()
     38 
     39     def set_param(self):
---> 40         for f,h in zip(self.sched_funcs,self.opt.hypers):
     41             h[self.pname] = f(self.n_epochs/self.epochs)
     42 

ipdb> self.sched_funcs
<function combine_scheds.<locals>._inner at 0x7ff2a5144e18>
ipdb> q


In [ ]:
ParamScheduler??

## Export

In [ ]:
!./notebook2script.py 11_train_imagenette.ipynb

Converted 11_train_imagenette.ipynb to exp/nb_11.py
